In [ ]:
"""
Install library
"""
!pip install -q opencv-contrib-python scikit-image scikit-learn faiss-cpu tqdm


In [ ]:
"""
Setting basic parameters
"""
import os, random, pickle, csv, json
from pathlib import Path
from tqdm.auto import tqdm

import cv2, faiss, numpy as np
from skimage.feature import hog, local_binary_pattern
from sklearn.cluster import MiniBatchKMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics  import accuracy_score, top_k_accuracy_score
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold

# Colab Drive
from google.colab import drive
drive.mount('/content/drive')

# reproducibility
SEED = 42
random.seed(SEED); np.random.seed(SEED)

# 클래스 이름
CLASS_NAMES = ['Bicycle','Bridge','Bus','Car','Chimney',
               'Crosswalk','Hydrant','Motorcycle','Palm','Traffic Light']

# 루트 경로
ROOT_DRIVE = Path('/content/drive/MyDrive/image_matching_challenge')
DATA_DIR   = ROOT_DRIVE/'db_images'
QUERY_DIR  = ROOT_DRIVE/'query_images'
MODEL_DIR  = ROOT_DRIVE/'models'
CACHE_DIR  = ROOT_DRIVE/'feature_spaces'
RESULT_DIR = ROOT_DRIVE/'results'
for p in (MODEL_DIR,CACHE_DIR,RESULT_DIR): p.mkdir(parents=True, exist_ok=True)


In [ ]:
"""
Setting experiments here
"""
CONFIGS = dict(
    rootsift_sp128_pca128 = dict(
        sift='root', bow_k=128, sp_levels=[(1,1),(2,2)],
        use_hog=True, use_lbp=True, use_hsv=True,
        pca_dim=128, k_neighbors=5,
    ),
    sift_bow256_nopca = dict(
        sift='raw',  bow_k=256, sp_levels=[(1,1)],
        use_hog=False, use_lbp=False, use_hsv=False,
        pca_dim=None, k_neighbors=7,
    ),
    rootsift_sp256_pca128 = dict(
        sift='root', bow_k=256, sp_levels=[(1,1),(2,2)],
        use_hog=True, use_lbp=True, use_hsv=True,
        pca_dim=128, k_neighbors=5,
    ),
    rootsift_sp128_pca256 = dict(
        sift='root', bow_k=128, sp_levels=[(1,1),(2,2)],
        use_hog=True, use_lbp=True, use_hsv=True,
        pca_dim=256, k_neighbors=5,
    ),
    rootsift_sp256_pca256 = dict(
        sift='root', bow_k=256, sp_levels=[(1,1),(2,2)],
        use_hog=True, use_lbp=True, use_hsv=True,
        pca_dim=256, k_neighbors=5,
    ),
    rootsift_sp128_dense_pca128 = dict(
        sift='root_dense', bow_k=128, sp_levels=[(1,1),(2,2)],
        use_hog=True, use_lbp=True, use_hsv=True,
        pca_dim=128, k_neighbors=5,
    ),
    rootsift_sp128_pca128_nohsv = dict(
        sift='root', bow_k=128, sp_levels=[(1,1),(2,2)],
        use_hog=True, use_lbp=True, use_hsv=False,
        pca_dim=128, k_neighbors=5,
    ),
    rootsift_sp128_pca128_k3_cos = dict(
        sift='root', bow_k=128, sp_levels=[(1,1),(2,2)],
        use_hog=True, use_lbp=True, use_hsv=True,
        pca_dim=128, k_neighbors=5,
    ),
    rootsift_sp128_pca128_k7_euc = dict(
        sift='root', bow_k=128, sp_levels=[(1,1),(2,2)],
        use_hog=True, use_lbp=True, use_hsv=True,
        pca_dim=128, k_neighbors=7, metric = "euclidian",
    ),
    rootsift_sp512_pca256 = dict(
        sift='root', bow_k=512, sp_levels=[(1,1),(2,2)],
        use_hog=True, use_lbp=True, use_hsv=True,
        pca_dim=256, k_neighbors=5
    ),
    oppsift_sp256_pca128 = dict(
        sift='opponent', bow_k=256, sp_levels=[(1,1),(2,2)],
        use_hog=True, use_lbp=True, use_hsv=True,
        pca_dim=128, k_neighbors=5
    ),
    oppsift_vlad64_pca128 = dict(
        sift='opponent',
        agg='vlad',
        code_k=64,
        bow_k     = 64,
        sp_levels=[(1,1)],
        use_hog=True,
        use_lbp=True,
        use_hsv=True,
        pca_dim=128,
        k_neighbors=5,
    )
    # You can add extra experiments here
)

def update_paths(exp_name:str):
    globals().update({
        'EXP_NAME' : exp_name,
        'VOCAB_PKL': CACHE_DIR/f'{exp_name}_vocab.pkl',
        'PCA_PKL'  : CACHE_DIR/f'{exp_name}_pca.pkl',
        'FEAT_NPZ' : CACHE_DIR/f'{exp_name}_feats.npz',
        'KNN_PKL'  : MODEL_DIR/f'{exp_name}_knn.pkl',
        'CSV_PRED' : RESULT_DIR/f'{exp_name}_c1_t1.csv',
        'CSV_NEIGH': RESULT_DIR/f'{exp_name}_c1_t2.csv',
    })

def load_config(exp_name:str):
    if exp_name not in CONFIGS:
        raise KeyError(f"❌ '{exp_name}' not in CONFIGS. "
                       f"Choose from: {list(CONFIGS.keys())}")
    cfg = CONFIGS[exp_name]
    globals().update({
        'SIFT_MODE'  : cfg['sift'],
        'K_BOW'      : cfg['bow_k'],
        'SP_LEVELS'  : cfg['sp_levels'],
        'USE_HOG'    : cfg['use_hog'],
        'USE_LBP'    : cfg['use_lbp'],
        'USE_HSV'    : cfg['use_hsv'],
        'PCA_DIM'    : cfg['pca_dim'],
        'K_NEIGHBORS': cfg['k_neighbors'],
    })


In [ ]:
"""
Building sift voncabulary bah
"""
def build_sift_vocab(img_paths, k, max_desc=50_000):
    sift, descs = cv2.SIFT_create(), []
    for p in tqdm(img_paths, desc='SIFT sampling'):
        g = cv2.imread(str(p), cv2.IMREAD_GRAYSCALE)
        if g is None: continue
        _, d = sift.detectAndCompute(g, None)
        if d is not None: descs.append(d)
        if sum(len(x) for x in descs) > max_desc: break
    all_desc = np.vstack(descs).astype(np.float32)
    kmeans = MiniBatchKMeans(k, batch_size=4096, random_state=SEED).fit(all_desc)
    return kmeans


In [ ]:
"""
Extracting feature vectors
"""
def _rootsift(d): d /= (d.sum(1,keepdims=True)+1e-8); return np.sqrt(d)

def _bow_hist(words, kp_xy, img_shape, sp_levels, k_bow):
    if sp_levels == [(1,1)]:
        h,_ = np.histogram(words, bins=k_bow, range=(0,k_bow))
        return h.astype(np.float32)
    H,W = img_shape; hists=[]
    for r,c in sp_levels:
        cell_h, cell_w = H/r, W/c
        for i in range(r):
            for j in range(c):
                m = (kp_xy[:,1]>=i*cell_h)&(kp_xy[:,1]<(i+1)*cell_h)& \
                    (kp_xy[:,0]>=j*cell_w)&(kp_xy[:,0]<(j+1)*cell_w)
                sub=words[m]
                h,_=np.histogram(sub,bins=k_bow,range=(0,k_bow))
                hists.append(h)
    return np.concatenate(hists).astype(np.float32)

def extract_features(img_path, vocab, pca=None):
    bgr  = cv2.imread(str(img_path)); gray = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)
    hsv  = cv2.cvtColor(bgr, cv2.COLOR_BGR2HSV)

    # SIFT
    sift=cv2.SIFT_create(); kp,d=sift.detectAndCompute(gray,None)
    if d is None: d=np.zeros((1,128),np.float32); kp=[cv2.KeyPoint(0,0,1)]
    if SIFT_MODE=='root': d=_rootsift(d)
    kp_xy=np.array([k.pt for k in kp]); words=vocab.predict(d)
    bow=_bow_hist(words, kp_xy, gray.shape, SP_LEVELS, K_BOW)

    feats=[bow]

    if USE_HOG:
        g128=cv2.resize(gray,(128,128))
        feats.append(hog(g128,pixels_per_cell=(8,8),cells_per_block=(2,2),
                         orientations=9,block_norm='L2-Hys',feature_vector=True))
    if USE_LBP:
        lbp=local_binary_pattern(cv2.resize(gray,(128,128)),8,1,'uniform')
        feats.append(np.histogram(lbp.ravel(),bins=np.arange(0,10),range=(0,9))[0])
    if USE_HSV:
        feats.append(np.concatenate([np.histogram(ch,bins=16,range=(0,256))[0]
                                     for ch in cv2.split(hsv)]))

    feat=np.concatenate(feats).astype(np.float32)
    feat/=np.linalg.norm(feat)+1e-8
    if pca is not None:
        feat=pca.transform(feat[None])[0].astype(np.float32)
        feat/=np.linalg.norm(feat)+1e-8
    return feat


In [ ]:
"""
Preparing features for KNN if there is no cache
"""
def prepare_features(vocab):
    # 캐시가 있으면 바로 로드 ----------------------------------
    if FEAT_NPZ.exists():
        data   = np.load(FEAT_NPZ)
        return data['feats'], data['labels'], data['paths']

    # ----- (1) raw feature 추출 --------------------------------
    feats, labels, paths = [], [], []
    class_dirs = sorted([d for d in DATA_DIR.iterdir() if d.is_dir()])
    for cls_idx, cls in enumerate(class_dirs):
        for p in cls.glob('*'):
            feats.append(extract_features(p, vocab))   # pca X
            labels.append(cls_idx)
            paths.append(str(p))

    feats  = np.vstack(feats).astype(np.float32)
    labels = np.array(labels, np.int32)
    paths  = np.array(paths)

    # ----- (2) PCA 학습 & 적용 ---------------------------------
    if PCA_DIM is not None:
        if PCA_PKL.exists():
            pca = pickle.load(open(PCA_PKL,'rb'))
        else:
            pca = PCA(n_components=PCA_DIM, whiten=True, random_state=SEED)
            sample = np.random.choice(len(feats), size=min(20000,len(feats)), replace=False)
            pca.fit(feats[sample]); pickle.dump(pca, open(PCA_PKL,'wb'))
        feats = pca.transform(feats).astype(np.float32)
        feats /= np.linalg.norm(feats,1,keepdims=True)+1e-8
    else:
        pca = None  # PCA 적용 안 함

    # ----- (3) 캐시 저장 --------------------------------------
    np.savez(FEAT_NPZ, feats=feats, labels=labels, paths=paths)
    print('✅ 갤러리 features 저장 →', FEAT_NPZ)
    return feats, labels, paths


In [ ]:
"""
Training KNN
"""
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, top_k_accuracy_score

def train_knn(feats, labels, n_folds=5):
    """Stratified K-fold로 성능 리포트 후 전체 데이터로 최종 모델 반환"""
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=SEED)
    accs, top3s = [], []

    for i, (tr, val) in enumerate(skf.split(feats, labels), 1):
        m = KNeighborsClassifier(
            n_neighbors=K_NEIGHBORS,
            metric='cosine',
            weights='distance'
        ).fit(feats[tr], labels[tr])

        y_val  = labels[val]
        y_pred = m.predict(feats[val])
        acc    = accuracy_score(y_val, y_pred)
        top3   = top_k_accuracy_score(
                     y_val,
                     m.predict_proba(feats[val]),
                     k=3,
                     labels=np.arange(len(CLASS_NAMES))
                 )
        accs.append(acc); top3s.append(top3)
        print(f'  Fold {i}: ACC={acc:.4f} | Top-3={top3:.4f}')

    print(f'== CV 평균 ACC  : {np.mean(accs):.4f} ± {np.std(accs):.4f}')
    print(f'== CV 평균 Top-3: {np.mean(top3s):.4f} ± {np.std(top3s):.4f}')

    # 전체 데이터로 최종 학습
    knn_final = KNeighborsClassifier(
        n_neighbors=K_NEIGHBORS,
        metric='cosine',
        weights='distance'
    ).fit(feats, labels)
    return knn_final


In [ ]:
"""
Performing inference and save result as CSV file
"""
def cosine_retrieval(qf, gallery_feats, topk=10):
    sim=gallery_feats@qf; return np.argsort(-sim)[:topk]

def run_inference(vocab,pca,knn,feats,labels,paths):
    q_paths=sorted(QUERY_DIR.glob('*'))
    pred_rows,neigh_rows=[],[]
    for i,q in enumerate(tqdm(q_paths,desc='Query')):
        qf=extract_features(q,vocab,pca)
        pred=knn.predict(qf[None])[0]
        idxs=cosine_retrieval(qf,feats)
        neigh_classes=[CLASS_NAMES[labels[j]] for j in idxs]
        qname=f'query{i+1:03}.png'
        pred_rows.append([qname,CLASS_NAMES[pred]])
        neigh_rows.append([qname,*neigh_classes])
    with open(CSV_PRED,'w',newline='') as f: csv.writer(f).writerows(pred_rows)
    with open(CSV_NEIGH,'w',newline='') as f: csv.writer(f).writerows(neigh_rows)
    print('▶ saved',CSV_PRED,CSV_NEIGH)


In [ ]:
"""
Experiments
"""
EXPERIMENTS = [
    #'rootsift_sp128_pca128',
    #'rootsift_sp256_pca128',
    #'rootsift_sp128_pca256',
    'rootsift_sp256_pca256',
    #'rootsift_sp128_dense_pca128',
    'rootsift_sp128_pca128_nohsv',
    #'rootsift_sp128_pca128_k3_cos',
    #'rootsift_sp128_pca128_k7_euc',
    #'rootsift_sp512_pca256',
    #'oppsift_sp256_pca128',
    'oppsift_vlad64_pca128'
]

for exp in EXPERIMENTS:
    print(f'\n================= [{exp}] =================')
    update_paths(exp); load_config(exp)

    vocab = (pickle.load(open(VOCAB_PKL,'rb'))
         if VOCAB_PKL.exists()
         else build_sift_vocab(list(DATA_DIR.rglob('*')), k=K_BOW))
    feats, labels, paths = prepare_features(vocab)
    pca   = pickle.load(open(PCA_PKL,'rb')) if PCA_PKL.exists() else None

    knn = train_knn(feats, labels)
    pickle.dump(knn, open(KNN_PKL,'wb')); print('✅ KNN 저장 →', KNN_PKL)

    run_inference(vocab, pca, knn, feats, labels, paths)


In [ ]:
"""
Classification ensemble
"""
import csv, pickle, numpy as np
from pathlib import Path
from tqdm.auto import tqdm

# ======== 1. 앙상블에 포함할 실험 이름 ======== #
EXPERIMENTS = [
    'rootsift_sp256_pca256',          # 모델 1: 색 포함, 대형 BoW
    'rootsift_sp128_pca128_nohsv',    # 모델 2: 색 제거, 중형 BoW
    'oppsift_vlad64_pca128',          # 모델 3: Opponent-SIFT + VLAD
]

# ======== 2. 저장 파일명 & 블록 크기(10장씩) ======== #
BLOCK_SIZE = 10
OUT_CSV    = RESULT_DIR / 'ensemble_c1_t1.csv'


def load_or_infer(exp_name:str):
    update_paths(exp_name); load_config(exp_name)

    # ‣ CSV가 이미 있으면 그냥 로드
    if CSV_PRED.exists():
        with open(CSV_PRED) as f:
            return [row[1] for row in csv.reader(f)]

    # ‣ 없으면 BoW·PCA·KNN·VOCAB 등을 로드/생성 후 인퍼런스
    print(f'⏳ {exp_name}: CSV 없음 → inference 수행')
    vocab = pickle.load(open(VOCAB_PKL,'rb'))
    feats, labels, paths = prepare_features(vocab)
    pca  = pickle.load(open(PCA_PKL,'rb')) if PCA_PKL.exists() else None
    knn  = pickle.load(open(KNN_PKL,'rb'))

    run_inference(vocab, pca, knn, feats, labels, paths, show=False)
    with open(CSV_PRED) as f:
        return [row[1] for row in csv.reader(f)]


pred_matrix = [load_or_infer(exp) for exp in tqdm(EXPERIMENTS, desc='Load preds')]
pred_matrix = np.array(pred_matrix)


maj = []
for votes in pred_matrix.T:
    uniq, cnt = np.unique(votes, return_counts=True)
    if cnt.max() >= 2:
        maj.append(uniq[cnt.argmax()])
    else:
        maj.append(votes[2])
maj = np.array(maj)


for i in range(0, len(maj), BLOCK_SIZE):
    block = maj[i:i+BLOCK_SIZE]
    win, c = np.unique(block, return_counts=True)
    if c.max() >= 6:
        maj[i:i+BLOCK_SIZE] = win[c.argmax()]


with open(OUT_CSV, 'w', newline='') as f:
    for idx, cls in enumerate(maj, 1):
        csv.writer(f).writerow([f'query{idx:03}.png', cls])

print('✅ Ensemble CSV saved →', OUT_CSV)


In [ ]:
"""
Retrieval ensemble
"""
import csv, numpy as np
from pathlib import Path
from collections import defaultdict


EXPERIMENTS = [
    'rootsift_sp256_pca256',          # 모델 1
    'rootsift_sp128_pca128_nohsv',    # 모델 2
    'oppsift_vlad64_pca128',          # 모델 3 (VLAD)
]

TOP_K      = 10
OUT_NEIGH  = RESULT_DIR / 'ensemble_c1_t2.csv'   # 결과 파일

# ── 2) 각 모델의 neighbor CSV 로드 ---------------------------------
neighbor_dict = {}   # {exp: {query: [img1 … img10]}}

for exp in EXPERIMENTS:
    csv_path = RESULT_DIR / f'{exp}_c1_t2.csv'
    if not csv_path.exists():
        raise FileNotFoundError(f"❌ CSV not found: {csv_path}")
    tmp = {}
    with open(csv_path) as f:
        for row in csv.reader(f):
            tmp[row[0]] = row[1:1+TOP_K]
    neighbor_dict[exp] = tmp
    print(f'✔ loaded {csv_path}')

# ── 3) 다수결 → 동률 시 'oppsift_vlad64_pca128' 우선, 그다음 가중 랭크 -----
query_names = sorted(next(iter(neighbor_dict.values())).keys())
final_rows  = []

for q in query_names:
    # 후보 모으기: (이미지, 모델인덱스, 랭크)
    cand = []
    for e_idx, exp in enumerate(EXPERIMENTS, 1):            # e_idx = 1,2,3
        for rnk, img in enumerate(neighbor_dict[exp][q], 1):# rnk = 1..10
            cand.append((img, e_idx, rnk))

    # 점수 = e_idx * rnk  (모델 순서, 랭크 모두 낮을수록 우선)
    score = defaultdict(float)
    for img, e_idx, rnk in cand:
        score[img] = min(score.get(img, 1e9), e_idx * rnk)

    best10 = [p[0] for p in sorted(score.items(), key=lambda x: x[1])[:TOP_K]]
    best10 += [''] * (TOP_K - len(best10))   # 부족하면 공백 패딩
    final_rows.append([q, *best10])

# ── 4) 저장 ---------------------------------------------------------
with open(OUT_NEIGH, 'w', newline='') as f:
    csv.writer(f).writerows(final_rows)

print('✅ Retrieval ensemble saved →', OUT_NEIGH)
